In [3]:
%tensorflow_version 1.x
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

TensorFlow 1.x selected.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
try:
    import keras
except:
    !pip install keras

Using TensorFlow backend.


In [5]:
from pathlib import Path
import os
os.chdir('/content/drive/My Drive/haikurnn-master/haikurnn-master/notebooks/models/v1')
tf_session = tf.Session()
from keras import backend as K
K.set_session(tf_session)
import math
from keras.callbacks import ModelCheckpoint,  CSVLogger
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
import requests
import numpy as np
import pandas as pd
from sklearn.externals import joblib

# Local library with model definitions for training and generating
from models import create_training_model

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Load Input

In [10]:
# Settings

# Percent of samples to use for training, might be necessary if you're running out of memory
sample_size = 1

# The latent dimension of the LSTM
latent_dim = 4096

# Number of epochs to train for
epochs = 40

root_path = Path('../../..')
input_path = root_path / 'input'
poem_path = input_path / 'poems'
haiku_path = poem_path / 'haikus.csv'

name = 'all_data_test_2'
output_dir = Path('output_%s' % name)
output_dir.mkdir()

In [11]:
f1 = open("/content/drive/My Drive/haikurnn-master/haikurnn-master/notebooks/models/v1/first.txt","r")
f2 = open("/content/drive/My Drive/haikurnn-master/haikurnn-master/notebooks/models/v1/second.txt","r")
f3 = open("/content/drive/My Drive/haikurnn-master/haikurnn-master/notebooks/models/v1/third.txt","r")

In [12]:
first = f1.read().split(sep='\n')
second = f2.read().split(sep='\n')
third = f3.read().split(sep='\n')

In [13]:
for x in first:
  if len(x) < 2:
    first.remove(x)
for x in second:
  if len(x) < 2:
    second.remove(x)
for x in third:
  if len(x) < 2:
    third.remove(x)

In [14]:
print(len(first), len(second), len(third))

0 3810 3810


In [25]:
def get_hyp_lm_tercets(tercets):
    new_tercets = []
    for tercet in tercets:
        new_tercets.append([])
        for verse in tercet:
            new_tercets[-1].append([])
            for hyp_w in verse:
                new_tercets[-1][-1].extend(hyp_w)
                new_tercets[-1][-1].append('<SEP>')
            new_tercets[-1][-1] = new_tercets[-1][-1][:-1]

    return new_tercets

def is_vowel(c):
    return c in 'aeiouAEIOUàìíèéùúüòï'

def unsplittable_cons():
    u_cons = []
    for c1 in ('b', 'c', 'd', 'f', 'g', 'p', 't', 'v'):
        for c2 in ('l', 'r'):
            u_cons.append(c1 + c2)

    others = ['gn', 'gh', 'ch']
    u_cons.extend(others)
    return u_cons


def are_cons_to_split(c1, c2):
    to_split = ('cq', 'cn', 'lm', 'rc', 'bd', 'mb', 'mn', 'ld', 'ng', 'nd', 'tm', 'nv', 'nc', 'ft', 'nf', 'gm', 'fm', 'rv', 'fp')
    return (c1 + c2) in to_split or (not is_vowel(c1) and (c1 == c2)) or ((c1 + c2) not in unsplittable_cons()) and (
        (not is_vowel(c1)) and (not is_vowel(c2)) and c1 != 's')


def is_diphthong(c1, c2):
    return (c1 + c2) in ('ia', 'ie', 'io', 'iu', 'ua', 'ue', 'uo', 'ui', 'ai', 'ei', 'oi', 'ui', 'au', 'eu', 'ïe', 'iú', 'iù')


def is_triphthong(c1, c2, c3):
    return (c1 + c2 + c3) in ('iai', 'iei', 'uoi', 'uai', 'uei', 'iuo')


def is_toned_vowel(c):
    return c in 'àìèéùòï'

def has_vowels(sy):
    for c in sy:
        if is_vowel(c):
            return True
    return False


def hyphenation(word):
    """
    Split word in syllables
    :param word: input string
    :return: a list containing syllables of the word
    """
    if not word or word == '':
        return []
    # elif len(word) == 3 and (is_vowel(word[1]) and is_vowel(word[2]) and not is_toned_vowel(word[2]) and (
    #     not is_diphthong(word[1], word[2]) or (word[1] == 'i'))):
    elif len(word) == 3 and (is_vowel(word[1]) and is_vowel(word[2]) and not is_toned_vowel(word[2]) and (
        not is_diphthong(word[1], word[2]))):
        return [word[:2]] + [word[2]]
    elif len(word) == 3 and is_vowel(word[0]) and not is_vowel(word[1]) and is_vowel(word[2]):
        return [word[:2]] + [word[2]]
    elif len(word) == 3:
        return [word]

    syllables = []
    is_done = False
    count = 0
    while not is_done and count <= len(word) - 1:
        syllables.append('')
        c = word[count]
        while not is_vowel(c) and count < len(word) - 1:
            syllables[-1] = syllables[-1] + c
            count += 1
            c = word[count]

        syllables[-1] = syllables[-1] + word[count]

        if count == len(word) - 1:
            is_done = True
        else:
            count += 1

            if count < len(word) and not is_vowel(word[count]):
                if count == len(word) - 1:
                    syllables[-1] += word[count]
                    count += 1
                elif count + 1 < len(word) and are_cons_to_split(word[count], word[count + 1]):
                    syllables[-1] += word[count]
                    count += 1
                elif count + 2 < len(word) and not is_vowel(word[count + 1]) and not is_vowel(word[count + 2]) and word[
                    count] != 's':
                    syllables[-1] += word[count]
                    count += 1
            elif count < len(word):
                if count + 1 < len(word) and is_triphthong(word[count - 1], word[count], word[count + 1]):
                    syllables[-1] += word[count] + word[count + 1]
                    count += 2
                elif is_diphthong(word[count - 1], word[count]):
                    syllables[-1] += word[count]
                    count += 1

                if count + 1 < len(word) and are_cons_to_split(word[count], word[count + 1]):
                    syllables[-1] += word[count]
                    count += 1

            else:
                is_done = True

    if not has_vowels(syllables[-1]) and len(syllables) > 1:
        syllables[-2] = syllables[-2] + syllables[-1]
        syllables = syllables[:-1]

    return syllables



def get_dc_hyphenation(canti):
    hyp_canti, hyp_tokens = [], []
    for canto in canti:
        hyp_canti.append([])
        for verso in canto:
            syllables = seq_hyphentation(verso)
            hyp_canti[-1].append(syllables)
            for syllable in syllables:
                hyp_tokens.extend(syllable)

    return hyp_canti, hyp_tokens


def seq_hyphentation(words):
    """
    Converts words in a list of strings into lists of syllables
    :param words: a list of words (strings)
    :return: a list of lists containing word syllables
    """
    return [hyphenation(w) for w in words]


def get_dc_cantos(filename, encoding=None):
    # raw_data = read_words(filename=filename)
    cantos, words, raw = [], [], []
    with open(filename, "r", encoding=encoding) as f:
        for line in f:
            sentence = line.strip()
            sentence = str.replace(sentence, "\.", " \. ")
            sentence = str.replace(sentence, "[", '')
            sentence = str.replace(sentence, "]", '')
            sentence = str.replace(sentence, "-", '')
            sentence = str.replace(sentence, ";", " ; ")
            sentence = str.replace(sentence, ",", " , ")
            # sentence = str.replace(sentence, " \'", '')
            sentence = str.replace(sentence, "\'", ' \' ')
            if len(sentence) > 1:
                # sentence = sentence.translate(string.punctuation)
                tokenized_sentence = nl.word_tokenize(sentence)
                # tokenized_sentence = sentence.split()
                tokenized_sentence = [w.lower() for w in tokenized_sentence if len(w) > 0]
                tokenized_sentence = [w for w in tokenized_sentence if "," not in w]
                tokenized_sentence = [w for w in tokenized_sentence if "." not in w]
                tokenized_sentence = [w for w in tokenized_sentence if ":" not in w]
                tokenized_sentence = [w for w in tokenized_sentence if ";" not in w]
                tokenized_sentence = [w for w in tokenized_sentence if "«" not in w]
                tokenized_sentence = [w for w in tokenized_sentence if "»" not in w]
                # ts = []
                ts = tokenized_sentence
                # [ts.extend(re.split("(\')", e)) for e in tokenized_sentence]
                tokenized_sentence = [w for w in ts if len(w) > 0]

                if len(tokenized_sentence) == 2:
                    cantos.append([])
                    raw.append([])
                elif len(tokenized_sentence) > 2:
                    raw[-1].append(sentence)
                    cantos[-1].append(tokenized_sentence)
                    words.extend(tokenized_sentence)
    print(cantos)
    return cantos, words, raw


def create_tercets(cantos):
    tercets = []
    for i,canto in enumerate(cantos):
        for v,verse in enumerate(canto):
            if v%3 == 0:
                tercets.append([])

            tercets[-1].append(verse)
        tercets = tercets[:-1]  # removes the last malformed tercets (only 2 verses)

    return tercets

def pad_list(l, pad_token, max_l_size, keep_lasts=False, pad_right=True):
    """
    Adds a padding token to a list
    inputs:
    :param l: input list to pad.
    :param pad_token: value to add as padding.
    :param max_l_size: length of the new padded list to return,
    it truncates lists longer that 'max_l_size' without adding
    padding values.
    :param keep_lasts: If True, preserves the max_l_size last elements
    of a sequence (by keeping the same order).  E.g.:
    if keep_lasts is True and max_l_size=3 [1,2,3,4] becomes [2,3,4].


    :return: the list padded or truncated.
    """
    to_pad = []
    max_l = min(max_l_size, len(l))  # maximum len
    l_init = len(l) - max_l if len(l) > max_l and keep_lasts else 0  # initial position where to sample from the list
    l_end = len(l) if len(l) > max_l and keep_lasts else max_l
    for i in range(l_init, l_end):
        to_pad.append(l[i])

    # for j in range(len(l), max_l_size):
    #     to_pad.append(pad_token)
    pad_tokens = [pad_token] * (max_l_size-len(l))
    padded_l = to_pad + pad_tokens if pad_right else pad_tokens + to_pad

    return padded_l


def save_data(data, file):
    with open(file, 'wb') as output:
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)

def load_data(file):
    with open(file, 'rb') as obj:
        return pickle.load(obj)

def print_and_write(file, s):
    print(s)
    file.write(s)


class Vocabulary(object):
    def __init__(self, vocab_size=None):
        self.dictionary = dict()
        self.rev_dictionary = dict()
        self.count = []
        self.special_tokens = []
        self.vocab_size = vocab_size

    def build_vocabulary_from_counts(self, count, special_tokens=[]):
        """
        Sets all the attributes of the Vocabulary object.
        :param count: a list of lists as follows: [['token', number_of_occurrences],...]
        :param special_tokens: a list of strings. E.g. ['<EOS>', '<PAD>',...]
        :return: None
        """

        dictionary = dict()
        for word, _ in count:
            dictionary[word] = len(dictionary)

        # adding eventual special tokens to the dictionary (e.g. <EOS>,<PAD> etc..)
        d = len(dictionary)
        for i, token in enumerate(special_tokens):
            dictionary[token] = d + i

        self.count = count
        self.dictionary = dictionary
        self.rev_dictionary = dict(zip(self.dictionary.values(), self.dictionary.keys()))
        self.special_tokens = special_tokens
        self.vocab_size = len(dictionary)

    def build_vocabulary_from_tokens(self, tokens, vocabulary_size=None, special_tokens=[]):
        """
        Given a list of tokens, it sets the Vocabulary object attributes by constructing
        a dictionary mapping each token to a unique id.
        :param tokens: a list of strings.
         E.g. ["the", "cat", "is", ... ".", "the", "house" ,"is" ...].
         NB: Here you should put all your token instances of the corpus.
        :param vocabulary_size: The number of elements of your vocabulary. If there are more
        than 'vocabulary_size' elements on tokens, it considers only the 'vocabulary_size'
        most frequent ones.
        :param special_tokens: Optional. A list of strings. Useful to add special tokens in vocabulary.
        If you don't have any, keep it empty.
        :return: None
        """

        vocabulary_size = vocabulary_size if vocabulary_size is not None else self.vocab_size
        vocabulary_size = vocabulary_size - (len(special_tokens) + 1) if vocabulary_size else None
        # counts occurrences of each token
        count = [['<UNK>', -1]]
        count.extend(collections.Counter(tokens).most_common(vocabulary_size))  # takes only the most frequent ones, if size is None takes them all
        self.build_vocabulary_from_counts(count, special_tokens)  # actually build the vocabulary
        self._set_unk_count(tokens)  # set the number of OOV instances

    @staticmethod
    def merge_vocabulary(vocab0, vocab1, vocabulary_size=-1):
        """
        Merge two Vocabulary objects into a new one.
        :param vocab0: first Vocabulary object
        :param vocab1: second Vocabulary object
        :param vocabulary_size: parameter to decide the merged vocabulary size.
        With default value -1, all the words of both vocabularies are preserved.
        When set to 0, the size of the vocabulary is set to the size of vocab0,
        when set to 1 it is kept the size of vocab1.
        :return: a new vocabulary
        """
        # get size of the new vocabulary
        vocab_size = vocab0.vocab_size + vocab1.vocab_size if vocabulary_size == -1 else vocabulary_size
        merged_special_tokens = list(set(vocab0.special_tokens) | set(vocab1.special_tokens))

        # merge the counts from the two vocabularies and then selects the most_common tokens
        merged_counts = collections.Counter(dict(vocab0.count)) + collections.Counter(dict(vocab1.count))
        merged_counts = merged_counts.most_common(vocab_size)
        count = [['<UNK>', -1]]
        count.extend(merged_counts)

        # create the new vocabulary
        merged_vocab = Vocabulary(vocab_size)
        merged_vocab.build_vocabulary_from_counts(count, merged_special_tokens)
        return merged_vocab

    @staticmethod
    def merge_vocabularies(vocab_list, vocab_size=None):
        """
        Join a list of vocabularies into a new one.
        :param vocab_list: a list of Vocabulary objects
        :param vocab_size: the maximum size of the merged vocabulary.
        :return: a vocabulary merging them all.
        """
        vocab_size = vocab_size if vocab_size else sum([v.vocab_size for v in vocab_list])
        merged_vocab = Vocabulary(vocab_size)
        for voc in vocab_list:
            merged_vocab = Vocabulary.merge_vocabulary(merged_vocab, voc, vocab_size)
        return merged_vocab

    def string2id(self, dataset):
        """
        Converts a dataset of strings into a dataset of ids according to the object dictionary.
        :param dataset: any string-based dataset with any nested lists.
        :return: a new dataset, with the same shape of dataset, where each string is mapped into its
        corresponding id associated in the dictionary (0 for unknown tokens).
        """

        def _recursive_call(items):
            new_items = []
            for item in items:
                if isinstance(item, str) or isinstance(item, int) or isinstance(item, float):
                    new_items.append(self.word2id(item))
                else:
                    new_items.append(_recursive_call(item))
            return new_items

        return _recursive_call(dataset)

    def id2string(self, dataset):
        """
        Converts a dataset of integer ids into a dataset of string according to the reverse dictionary.
        :param dataset: any int-based dataset with any nested lists. Allowed types are int, np.int32, np.int64.
        :return: a new dataset, with the same shape of dataset, where each token is mapped into its
        corresponding string associated in the reverse dictionary.
        """
        def _recursive_call(items):
            new_items = []
            for item in items:
                if isinstance(item, int) or isinstance(item, np.int) or isinstance(item, np.int32) or isinstance(item, np.int64):
                    new_items.append(self.id2word(item))
                else:
                    new_items.append(_recursive_call(item))
            return new_items

        return _recursive_call(dataset)

    def word2id(self, item):
        """
        Maps a string token to its corresponding id.
        :param item: a string.
        :return: If the token belongs to the vocabulary, it returns an integer id > 0, otherwise
        it returns the value associated to the unknown symbol, that is typically 0.
        """
        return self.dictionary[item] if item in self.dictionary else self.dictionary['<UNK>']

    def id2word(self, token_id):
        """
        Maps an integer token to its corresponding string.
        :param token_id: an integer.
        :return: If the id belongs to the vocabulary, it returns the string
        associated to it, otherwise it returns the string associated
        to the unknown symbol, that is '<UNK>'.
        """

        return self.rev_dictionary[token_id] if token_id in self.rev_dictionary else self.rev_dictionary[self.dictionary['<UNK>']]

    def get_unk_count(self):
        return self.count[0][1]

    def _set_unk_count(self, tokens):
        """
        Sets the number of OOV instances in the tokens provided
        :param tokens: a list of tokens
        :return: None
        """
        data = list()
        unk_count = 0
        for word in tokens:
            if word in self.dictionary:
                index = self.dictionary[word]
            else:
                index = 0  # dictionary['<UNK>']
                unk_count += 1
            data.append(index)
        self.count[0][1] = unk_count

    def add_element(self, name, is_special_token=False):
        if name not in self.dictionary:
            self.vocab_size += 1
            self.dictionary[name] = self.vocab_size
            self.rev_dictionary[self.vocab_size] = name

            if is_special_token:
                self.special_tokens = list(self.special_tokens)
                self.special_tokens.append(name)

            self.count.append([name, 1])

    def set_vocabulary(self, dictionary, rev_dictionary, special_tokens, vocab_size):
        self.dictionary = dictionary,
        self.rev_dictionary = rev_dictionary
        self.special_tokens = special_tokens
        self.vocab_size = vocab_size

    @staticmethod
    def load_vocabulary(filename):
        return load_data(filename)

    def save_vocabulary(self, filename):
        save_data(self, filename)

class SyLMDataset(object):
    def __init__(self, config, sy_vocab=None):
        self.config = config
        self.vocabulary = sy_vocab

        self.raw_train_x = []
        self.raw_val_x = []
        self.raw_test_x = []
        self.raw_x = []

        self.train_x, self.train_y = [], []
        self.val_x, self.val_y = [], []
        self.test_x, self.test_y = [], []
        self.x, self.y = [], []

    def initialize(self, sess):
        pass

    def load(self, sources):
        """
        Extract raw texts form sources and gather them all together.
        :param sources: a string or an iterable of strings containing the file(s)
        to process in order to build the dataset.
        :return: a list of raw strings.
        """
        return NotImplementedError

    def build(self, sources, split_size=0.8):
        """
        :param sources: a string or an iterable of strings containing the file(s)
        to process in order to build the dataset.
        :param split_size: the size to split the dataset, set >=1.0 to not split.
        """

        raw_x = self.load(sources)
        # raw_x = self.tokenize([self.preprocess(ex) for ex in raw_x])  # fixme
        # splitting data
        print(raw_x)
        self.raw_x = raw_x
        if split_size < 1.0:
            self.raw_train_x, self.raw_test_x = self.split(self.raw_x, train_size=split_size)
            self.raw_train_x, self.raw_val_x = self.split(self.raw_train_x, train_size=split_size)
        else:
            self.raw_train_x = self.raw_x

        if self.vocabulary is None:
            # creates vocabulary
            tokens = [item for sublist in self.raw_train_x for item in sublist]  # get tokens
            special_tokens = ("<GO>", "<PAD>", "<SEP>", "<EOS>", "<EOV>")
            self._create_vocab(tokens, special_tokens=special_tokens)

        # creates x,y for train
        self.train_x = self._build_dataset(self.raw_train_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.train_y = self._build_dataset(self.raw_train_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)

        # creates x,y for validation
        self.val_x = self._build_dataset(self.raw_val_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.val_y = self._build_dataset(self.raw_val_x, insert_go=False, max_len=self.config.sentence_max_len, shuffle=False)

        # creates x,y for validation
        self.test_x = self._build_dataset(self.raw_test_x, insert_go=True, max_len=self.config.sentence_max_len, shuffle=False)
        self.test_y = self._build_dataset(self.raw_test_x, insert_go=False, max_len=self.config.sentence_max_len, shuffle=False)

    def _create_vocab(self, tokens, special_tokens=("<PAD>", "<GO>", "<SEP>", "<EOV>", "<EOS>")):
        """
        Create the vocabulary. Special tokens can be added to the tokens obtained from
        the corpus.
        :param tokens: a list of all the tokens in the corpus. Each token is a string.
        :param special_tokens: a list of strings.
        """
        vocab = Vocabulary(vocab_size=self.config.input_vocab_size)
        vocab.build_vocabulary_from_tokens(tokens, special_tokens=special_tokens)
        self.vocabulary = vocab

    @staticmethod
    def split(raw_data, train_size=0.8):
        size = math.floor(len(raw_data)*train_size)
        return raw_data[:size], raw_data[size:]

    @staticmethod
    def preprocess(txt):
        return txt

    @staticmethod
    def shuffle(x):
        return random.sample(x, len(x))

    @staticmethod
    def tokenize(txt):
        return txt

    def _build_dataset(self, raw_data, max_len=100, insert_go=True, keep_lasts=False, pad_right=True, shuffle=True):
        """
        Converts all the tokens in e1_raw_data by mapping each token with its corresponding
        value in the dictionary. In case of token not in the dictionary, they are assigned to
        a specific id. Each sequence is padded up to the seq_max_len setup in the config.

        :param raw_data: list of sequences, each sequence is a list of tokens (strings).
        :param max_len: max length of a sequence, crop longer and pad smaller ones.
        :param insert_go: True to insert <GO>, False otherwise.
        :param keep_lasts: True to truncate initial elements of a sequence.
        :param pad_right: pad to the right (default value True), otherwise pads to left.
        :param shuffle: Optional. If True data are shuffled.
        :return: A list of sequences where each token in each sequence is an int id.
        """
        dataset = []
        for sentence in raw_data:
            sentence_ids = [self.vocabulary.word2id("<GO>")] if insert_go else []
            sentence_ids.extend([self.vocabulary.word2id(w) for w in sentence])
            sentence_ids.append(self.vocabulary.word2id("<EOS>"))
            #sentence_ids = pad_list(sentence_ids, self.vocabulary.word2id("<PAD>"), max_len, keep_lasts=keep_lasts, pad_right=pad_right)

            dataset.append(sentence_ids)

        if shuffle:
            return random.sample(dataset, len(dataset))
        else:
            return dataset

    def get_batches(self, batch_size=32):
        """
        Iterator over the training set. Useful method to run experiments.
        :param batch_size: size of the mini_batch
        :return: input and target.
        """
        x, y = self.train_x, self.train_y
        
        i = random.randint(0, batch_size)
        batches = []
        eov = self.vocabulary.word2id("<EOV>")
        # prepare batches
        while i < len(x):
            j = 0
            batch_x, batch_y = [], []
            while j < batch_size and i+j<len(x):
                for c in x[i+j]:
                  batch_x.append(c)
                batch_x.append(eov)
                for c in y[i+j]:
                  batch_y.append(c)
                batch_y.append(eov)
                j += 1
            i += batch_size

            batches.append((batch_x, batch_y))

        # shuffle
        random.shuffle(batches)

        # supply
        i = 0
        while i < len(batches):
            yield batches[i][0], batches[i][1]
            i += 1

class DanteSyLMDataset(SyLMDataset):
    def __init__(self, config, sy_vocab=None):
        """
        Class to create a dataset from Dante Alighieri's Divine Comedy.
        :param config: a Config object
        :param sy_vocab: (optional) a Vocabulary object where tokens of the dictionary
        are syllables. If None, the vocabulary is create automatically from the source.
        """
        super().__init__(config, sy_vocab)

    def load(self, sources):
        """
        Load examples from dataset
        :param sources: data filepath.
        :return:
        """
        canti, _, raw = get_dc_cantos(filename=sources)  # get raw data from file
        canti, tokens = get_dc_hyphenation(canti)  # converts each
        print(canti)
        tercets = create_tercets(canti)
        tercets = get_hyp_lm_tercets(tercets)
        print(tercets)
        x = []
        for tercet in tercets:
            x.append([])
            print(tercet)
            for verse in tercet:
                x[-1].extend(verse)
                x[-1].append("<EOV>")
        print(x)
        #x = self.shuffle(x)
        return x

def seq2str(seq):
    def output2string(batch, rev_vocabulary, special_tokens, end_of_tokens):
        to_print = ''
        for token in batch:
            if token in special_tokens:
                to_print += ' '
            elif end_of_tokens and token in end_of_tokens:
                to_print += '\n'
            elif token in rev_vocabulary:
                to_print += rev_vocabulary[token]
            else:
                to_print += '<UNK>'
        return to_print

    return output2string(seq, poetry_sy_lm_dataset.vocabulary.rev_dictionary,
      special_tokens=[poetry_sy_lm_dataset.vocabulary.word2id("<PAD>"), 0, poetry_sy_lm_dataset.vocabulary.word2id("<SEP>"),
                      poetry_sy_lm_dataset.vocabulary.word2id("<GO>"), poetry_sy_lm_dataset.vocabulary.word2id("<EOS>")],
      end_of_tokens=[poetry_sy_lm_dataset.vocabulary.word2id("<EOV>")])

class cnfg:
  vocab_size = 1884
  input_vocab_size = 1884
  emb_size = 300
  sentence_max_len = 75

In [26]:
import collections
import random
import nltk as nl
nl.download('punkt')
config = cnfg()
poetry_sy_lm_dataset = DanteSyLMDataset(config, sy_vocab=None)

data_path = os.path.join(os.getcwd(), "first.txt") 
poetry_sy_lm_dataset.build(data_path, split_size=0.9)  # actual creation of  vocabulary (if not provided) and dataset
print("Train size: " + str(len(poetry_sy_lm_dataset.train_y)))
print("Val size: " + str(len(poetry_sy_lm_dataset.val_y)))
print("Test size: " + str(len(poetry_sy_lm_dataset.test_y)))
first = [b for b in poetry_sy_lm_dataset.get_batches(20)]

poetry_sy_lm_dataset1 = DanteSyLMDataset(config, sy_vocab=poetry_sy_lm_dataset.vocabulary)

data_path = os.path.join(os.getcwd(), "second.txt")  # dataset location, here just the name of the source file
poetry_sy_lm_dataset.build(data_path, split_size=0.9)  # actual creation of  vocabulary (if not provided) and dataset
print("Train size: " + str(len(poetry_sy_lm_dataset.train_y)))
print("Val size: " + str(len(poetry_sy_lm_dataset.val_y)))
print("Test size: " + str(len(poetry_sy_lm_dataset.test_y)))
second = [b for b in poetry_sy_lm_dataset.get_batches(20)]

poetry_sy_lm_dataset2 = DanteSyLMDataset(config, sy_vocab=poetry_sy_lm_dataset1.vocabulary)

data_path = os.path.join(os.getcwd(), "third.txt")  # dataset location, here just the name of the source file
poetry_sy_lm_dataset.build(data_path, split_size=0.9)  # actual creation of  vocabulary (if not provided) and dataset
print("Train size: " + str(len(poetry_sy_lm_dataset.train_y)))
print("Val size: " + str(len(poetry_sy_lm_dataset.val_y)))
print("Test size: " + str(len(poetry_sy_lm_dataset.test_y)))
third = [b for b in poetry_sy_lm_dataset.get_batches(20)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[]
[]
[]
[]
[]
Train size: 0
Val size: 0
Test size: 0


IndexError: ignored

In [9]:
df = pd.DataFrame()
df['0'] = first
df['1'] = second
df['2'] = third
df['0_syllables'] = [11 for x in range(3810)]
df['1_syllables'] = [11 for x in range(3810)]
df['2_syllables'] = [11 for x in range(3810)]
df

,0,1,2,0_syllables,1_syllables,2_syllables
0,in tutte tue question certo mi piaci,rispuose ma ' l bollor dell ' acqua rossa,dovea ben solver l ' una che tu faci,11,11,11
1,letè vedrai ma fuor di questa fossa,là dove vanno l ' anime a lavarsi,quando la colpa pentuta è rimossa,11,11,11
2,poi disse omai è tempo da scostarsi,dal bosco fa che di retro a me vegne,li margini fan via che non son arsi,11,11,11
3,ora cen porta l ' un de ' duri margini,e ' l fummo del ruscel di sopra aduggia,sí che dal foco salva l ' acqua e li argini,11,11,11
4,quale i fiamminghi tra guizzante e bruggia,temendo il fiotto che ' nver lor s ' avventa,fanno lo schermo perché ' l mar si fuggia,11,11,11
...,...,...,...,...,...,...
3805,dentro dal ciel della divina pace,si gira un corpo nella cui virtute,l ' esser di tutto suo contento giace,11,11,11
3806,lo ciel seguente c ' ha tante vedute,quell ' esser parte per diverse essenze,da lui distinte e da lui contenute,11,11,11
3807,li altri giron per varie differenze,le distinzion che dentro da sé hanno,dispongono a lor fini e lor semenze,11,11,11
3808,questi organi del mondo cosí vanno,come tu vedi omai di grado in grado,che di su prendono e di sotto fanno,11,11,11


# Format Input for Training

In [10]:
# Drop samples that are longer that the 99th percentile of length

max_line_length = int(max([df['%s' % i].str.len().quantile(.99) for i in range(3)]))
df = df[
    (df['0'].str.len() <= max_line_length) & 
    (df['1'].str.len() <= max_line_length) & 
    (df['2'].str.len() <= max_line_length)
].copy()
df

,0,1,2,0_syllables,1_syllables,2_syllables
0,in tutte tue question certo mi piaci,rispuose ma ' l bollor dell ' acqua rossa,dovea ben solver l ' una che tu faci,11,11,11
1,letè vedrai ma fuor di questa fossa,là dove vanno l ' anime a lavarsi,quando la colpa pentuta è rimossa,11,11,11
2,poi disse omai è tempo da scostarsi,dal bosco fa che di retro a me vegne,li margini fan via che non son arsi,11,11,11
3,ora cen porta l ' un de ' duri margini,e ' l fummo del ruscel di sopra aduggia,sí che dal foco salva l ' acqua e li argini,11,11,11
4,quale i fiamminghi tra guizzante e bruggia,temendo il fiotto che ' nver lor s ' avventa,fanno lo schermo perché ' l mar si fuggia,11,11,11
...,...,...,...,...,...,...
3805,dentro dal ciel della divina pace,si gira un corpo nella cui virtute,l ' esser di tutto suo contento giace,11,11,11
3806,lo ciel seguente c ' ha tante vedute,quell ' esser parte per diverse essenze,da lui distinte e da lui contenute,11,11,11
3807,li altri giron per varie differenze,le distinzion che dentro da sé hanno,dispongono a lor fini e lor semenze,11,11,11
3808,questi organi del mondo cosí vanno,come tu vedi omai di grado in grado,che di su prendono e di sotto fanno,11,11,11


In [11]:
# Pad the lines to the max line length with new lines
for i in range(3):
    # For input, duplicate the first character
    # TODO - Why?
    df['%s_in' % i] = (df[str(i)].str[0] + df[str(i)]).str.pad(max_line_length+2, 'right', '\n')
    
    # 
    #df['%s_out' % i] = df[str(i)].str.pad(max_line_len, 'right', '\n') + ('\n' if i == 2 else df[str(i+1)].str[0])
    
    # TODO - trying to add the next line's first character before the line breaks
    if i == 2: # If it's the last line
        df['%s_out' % i] = df[str(i)].str.pad(max_line_length+2, 'right', '\n')
    else: 
        # If it's the first or second line, add the first character of the next line to the end of this line.
        # This helps with training so that the next RNN has a better chance of getting the first character right.
        df['%s_out' % i] = (df[str(i)] + '\n' + df[str(i+1)].str[0]).str.pad(max_line_length+2, 'right', '\n')
    
max_line_length += 2

df

,0,1,2,0_syllables,1_syllables,2_syllables,0_in,0_out,1_in,1_out,2_in,2_out
0,in tutte tue question certo mi piaci,rispuose ma ' l bollor dell ' acqua rossa,dovea ben solver l ' una che tu faci,11,11,11,in tutte tue question certo mi piaci\n\n\n\n...,in tutte tue question certo mi piaci\nr\n\n\n...,rrispuose ma ' l bollor dell ' acqua rossa\n\n...,rispuose ma ' l bollor dell ' acqua rossa\nd\n...,ddovea ben solver l ' una che tu faci\n\n\n\n\...,dovea ben solver l ' una che tu faci\n\n\n\n\n...
1,letè vedrai ma fuor di questa fossa,là dove vanno l ' anime a lavarsi,quando la colpa pentuta è rimossa,11,11,11,letè vedrai ma fuor di questa fossa\n\n\n\n\...,letè vedrai ma fuor di questa fossa\nl\n\n\n\...,llà dove vanno l ' anime a lavarsi\n\n\n\n\n\n...,là dove vanno l ' anime a lavarsi\nq\n\n\n\n\n...,qquando la colpa pentuta è rimossa\n\n\n\n\n\n...,quando la colpa pentuta è rimossa\n\n\n\n\n\n\...
2,poi disse omai è tempo da scostarsi,dal bosco fa che di retro a me vegne,li margini fan via che non son arsi,11,11,11,poi disse omai è tempo da scostarsi\n\n\n\n\...,poi disse omai è tempo da scostarsi\nd\n\n\n\...,ddal bosco fa che di retro a me vegne\n\n\n\n\...,dal bosco fa che di retro a me vegne\nl\n\n\n\...,lli margini fan via che non son arsi\n\n\n\n\n...,li margini fan via che non son arsi\n\n\n\n\n\...
3,ora cen porta l ' un de ' duri margini,e ' l fummo del ruscel di sopra aduggia,sí che dal foco salva l ' acqua e li argini,11,11,11,ora cen porta l ' un de ' duri margini\n\n\n...,ora cen porta l ' un de ' duri margini\ne\n\n...,ee ' l fummo del ruscel di sopra aduggia\n\n\n...,e ' l fummo del ruscel di sopra aduggia\ns\n\n...,ssí che dal foco salva l ' acqua e li argini\n...,sí che dal foco salva l ' acqua e li argini\n\...
4,quale i fiamminghi tra guizzante e bruggia,temendo il fiotto che ' nver lor s ' avventa,fanno lo schermo perché ' l mar si fuggia,11,11,11,quale i fiamminghi tra guizzante e bruggia\n...,quale i fiamminghi tra guizzante e bruggia\nt...,ttemendo il fiotto che ' nver lor s ' avventa\...,temendo il fiotto che ' nver lor s ' avventa\n...,ffanno lo schermo perché ' l mar si fuggia\n\n...,fanno lo schermo perché ' l mar si fuggia\n\n\...
...,...,...,...,...,...,...,...,...,...,...,...,...
3805,dentro dal ciel della divina pace,si gira un corpo nella cui virtute,l ' esser di tutto suo contento giace,11,11,11,dentro dal ciel della divina pace\n\n\n\n\n\...,dentro dal ciel della divina pace\ns\n\n\n\n\...,ssi gira un corpo nella cui virtute\n\n\n\n\n\...,si gira un corpo nella cui virtute\nl\n\n\n\n\...,ll ' esser di tutto suo contento giace\n\n\n\n...,l ' esser di tutto suo contento giace\n\n\n\n\...
3806,lo ciel seguente c ' ha tante vedute,quell ' esser parte per diverse essenze,da lui distinte e da lui contenute,11,11,11,lo ciel seguente c ' ha tante vedute\n\n\n\n...,lo ciel seguente c ' ha tante vedute\nq\n\n\n...,qquell ' esser parte per diverse essenze\n\n\n...,quell ' esser parte per diverse essenze\nd\n\n...,dda lui distinte e da lui contenute\n\n\n\n\n\...,da lui distinte e da lui contenute\n\n\n\n\n\n...
3807,li altri giron per varie differenze,le distinzion che dentro da sé hanno,dispongono a lor fini e lor semenze,11,11,11,li altri giron per varie differenze\n\n\n\n\...,li altri giron per varie differenze\nl\n\n\n\...,lle distinzion che dentro da sé hanno\n\n\n\n\...,le distinzion che dentro da sé hanno\nd\n\n\n\...,ddispongono a lor fini e lor semenze\n\n\n\n\n...,dispongono a lor fini e lor semenze\n\n\n\n\n\...
3808,questi organi del mondo cosí vanno,come tu vedi omai di grado in grado,che di su prendono e di sotto fanno,11,11,11,questi organi del mondo cosí vanno\n\n\n\n\n...,questi organi del mondo cosí vanno\nc\n\n\n\n...,ccome tu vedi omai di grado in grado\n\n\n\n\n...,come tu vedi omai di grado in grado\nc\n\n\n\n...,cche di su prendono e di sotto fanno\n\n\n\n\n...,che di su prendono e di sotto fanno\n\n\n\n\n\...


In [46]:
inputs = df[['0_in', '1_in', '2_in']].values

tokenizer = Tokenizer(filters='', char_level=True)
tokenizer.fit_on_texts(inputs.flatten())
n_tokens = len(tokenizer.word_counts) + 1

# X is the input for each line in sequences of one-hot-encoded values
X = np_utils.to_categorical([
    tokenizer.texts_to_sequences(inputs[:,i]) for i in range(3)
], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

# Y is the output for each line in sequences of one-hot-encoded values
Y = np_utils.to_categorical([
    tokenizer.texts_to_sequences(outputs[:,i]) for i in range(3)
], num_classes=n_tokens)

# X_syllables is the count of syllables for each line
X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

In [31]:
joblib.dump([latent_dim, n_tokens, max_line_length, tokenizer], str(output_dir / 'metadata.pkl'))

['output_all_data_test_2/metadata.pkl']

# Training Model

In [14]:
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.utils import np_utils
import numpy as np

In [15]:
class TrainingLine:
    def __init__(self, name, previous_line, lstm, n_tokens):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)

        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)

        #self.lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % name)

        if previous_line:
            initial_state = [
                Add(name='add_h_%s' % name)([
                    previous_line.lstm_h,
                    self.syllable_dense_output
                ]),
                Add(name='add_c_%s' % name)([
                    previous_line.lstm_c,
                    self.syllable_dense_output
                ])
            ]
        else:
            initial_state = [self.syllable_dense_output, self.syllable_dense_output]

        self.lstm_out, self.lstm_h, self.lstm_c = lstm(self.char_input, initial_state=initial_state)

        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

def custom_loss(line1, line2, line3):
    loss = 0
    if line1[-1] == line3[-1]:
        loss += .5
    else:
        loss -= .5
    return loss


def create_training_model(latent_dim, n_tokens):
    lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm')
    lines = []
    inputs = []
    outputs = []

    for i in range(3):
        previous_line = lines[-1] if lines else None
        lines.append(TrainingLine('line_%s' % i, previous_line, lstm, n_tokens))
        inputs += [lines[-1].char_input, lines[-1].syllable_input]
        outputs.append(lines[-1].output)

    training_model = Model(inputs, outputs)
    training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

    return training_model, lstm, lines, inputs, outputs


In [16]:
training_model, lstm, lines, inputs, outputs = create_training_model(latent_dim, n_tokens)

filepath = str(output_dir / ("%s-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5" % latent_dim))
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

csv_logger = CSVLogger(str(output_dir / 'training_log.csv'), append=True, separator=',')

callbacks_list = [checkpoint, csv_logger]

training_model.fit([
    X[0], X_syllables[:,0], 
    X[1], X_syllables[:,1], 
    X[2], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=epochs, validation_split=.05, callbacks=callbacks_list)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 3552 samples, validate on 187 samples
Epoch 1/40
3552/3552 [==============================] - 59s 17ms/step - loss: 8.7228 - output_line_0_loss: 2.9427 - output_line_1_loss: 2.9311 - output_line_2_loss: 2.8393 - val_loss: 8.0130 - val_output_line_0_loss: 2.7742 - val_output_line_1_loss: 2.6469 - val_output_line_2_loss: 2.5916

Epoch 00001: loss improved from inf to 8.72276, saving model to output_all_data_test_2/2048-01-8.72-8.01.hdf5
Epoch 2/40
3552/3552 [==============================] - 56s 16ms/step - loss: 7.5021 - output_line_0_loss: 2.4302 - output_line_1_loss: 2.5625 - output_line_2_loss: 2.5051 - val_loss: 6.8102 - val_output_line_0_loss: 2.2969 - val_output_line_1_loss: 2.2922 - val_output_line_2_loss: 2.2206

Epoch 00002: loss improved from 8.72276 to 7.50212, saving model to output_all_data_t

# Test Model

In [60]:
class GeneratorLine:
    def __init__(self, name, training_line, lstm, n_tokens):
        self.char_input = Input(shape=(None, n_tokens), name='char_input_%s' % name)

        self.syllable_input = Input(shape=(1,), name='syllable_input_%s' % name)
        self.syllable_dense = Dense(lstm.units, activation='relu', name='syllable_dense_%s' % name)
        self.syllable_dense_output = self.syllable_dense(self.syllable_input)

        self.h_input = Input(shape=(lstm.units,), name='h_input_%s' % name)
        self.c_input = Input(shape=(lstm.units,), name='c_input_%s' % name)
        initial_state = [self.h_input, self.c_input]

        self.lstm = lstm

        self.lstm_out, self.lstm_h, self.lstm_c = self.lstm(self.char_input, initial_state=initial_state)

        self.output_dense = Dense(n_tokens, activation='softmax', name='output_%s' % name)
        self.output = self.output_dense(self.lstm_out)

        self.syllable_dense.set_weights(training_line.syllable_dense.get_weights())
        #self.lstm.set_weights(lstm.get_weights())
        self.output_dense.set_weights(training_line.output_dense.get_weights())

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    # From https://github.com/llSourcell/keras_explained/blob/master/gentext.py
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

class Generator1:
    def __init__(self, lstm, lines, tf_session, tokenizer, n_tokens, max_line_length):
        self.tf_session = tf_session
        self.tokenizer = tokenizer
        self.n_tokens = n_tokens
        self.max_line_length = max_line_length

        self.lstm = LSTM(
            lstm.units, return_state=True, return_sequences=True,
            name='generator_lstm'
        )
        self.lines = [
            GeneratorLine(
                'generator_line_%s' % i,
                lines[i], self.lstm, self.n_tokens
            ) for i in range(3)
        ]
        self.lstm.set_weights(lstm.get_weights())

    def generate_haiku(self, syllables=[11, 11, 11], temperature=.7, first_char=None):
        output = []
        h = None
        c = None

        if first_char is None:
            first_char = chr(int(np.random.randint(ord('a'), ord('z')+1)))

        next_char = self.tokenizer.texts_to_sequences(first_char)[0][0]

        for i in range(3):
            line = self.lines[i]
            s = self.tf_session.run(
                line.syllable_dense_output,
                feed_dict={
                    line.syllable_input: [[syllables[i]]]
                }
            )

            if h is None:
                h = s
                c = s
            else:
                h = h + s
                c = c + s

            line_output = [next_char]
            tokens = self.n_tokens
            end = False
            next_char = None
            for i in range(self.max_line_length):
                char, h, c = self.tf_session.run(
                    [line.output, line.lstm_h, line.lstm_c],
                    feed_dict={
                        line.char_input: [[
                            np_utils.to_categorical(
                                line_output[-1],
                                num_classes=self.n_tokens
                            )
                        ]],
                        line.h_input: h,
                        line.c_input: c
                    }
                )

                char = sample(char[0,0], temperature)
                if char == 1 and not end:
                    end = True
                if char != 1 and end:
                    next_char = char
                    char = 1
                line_output.append(char)
            
            cleaned_text = self.tokenizer.sequences_to_texts([
                line_output
            ])[0].strip()[1:].replace(
                '   ', '\n'
            ).replace(' ', '').replace('\n', ' ')

            print(cleaned_text)
            output.append(cleaned_text)
        print('\n')
        return output

In [61]:
generator = Generator1(lstm, lines, tf_session, tokenizer, n_tokens, max_line_length)

In [67]:
for boh in range(30):
  generator.generate_haiku()


 sí che ' l pregno aere in acqua si converse
la pioggoa cappel du questo menso
di muïato na or di sta cotti ! '


 elli avean cappe con cappucci bassi
dinanzi alli occhi fatte della taglia
che in clugní per li monaci fassi


 l ' altro che già uscí preso di nave
veggio vender sua figlia e paggionte
come giusto pente av ' era punto




IndexError: ignored

In [59]:
print(giac[0])

 l ' amor ch ' ad esso troppo s ' abbandona


In [21]:
training_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
syllable_input_line_0 (InputLay (None, 1)            0                                            
__________________________________________________________________________________________________
char_input_line_0 (InputLayer)  (None, None, 43)     0                                            
__________________________________________________________________________________________________
syllable_dense_line_0 (Dense)   (None, 2048)         4096        syllable_input_line_0[0][0]      
__________________________________________________________________________________________________
syllable_input_line_1 (InputLay (None, 1)            0                                            
____________________________________________________________________________________________